In [15]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd

In [16]:
# Setup splinter # leave page open
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [/Users/naldrich/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [17]:
#Define url and create function
def create_soup(url):
    browser.visit(url)
    # Create BeautifulSoup object; parse with 'html.parser'
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [18]:
#Define the url to visit and call the function create_soup
url = 'https://redplanetscience.com/'
soup = create_soup(url)

In [19]:
# grab title
title = soup.find('div' , class_="content_title").text
title

"NASA's Mars 2020 Comes Full Circle"

In [20]:
# grab paragraph
news_p = soup.find('div', class_="article_teaser_body").text
news_p

"Aiming to pinpoint the Martian vehicle's center of gravity, engineers took NASA's 2,300-pound Mars 2020 rover for a spin in the clean room at JPL. "

In [21]:
# Define url and call create_soup function to return soup 
featured_url = 'https://spaceimages-mars.com/'
soup = create_soup(featured_url)

In [22]:
# Create BeautifulSoup object; parse with 'html.parser' to the second site
try:
    target = 'button[class="btn btn-outline-light"]'
    browser.find_by_tag(target).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_src = soup.find('img', class_="fancybox-image")['src']
except:
    print('can\'t find the image')
featured_image_url = featured_url + image_src

In [23]:
# define url to visit
mars_facts_url = 'https://galaxyfacts-mars.com/'
# using pandas read html
tables = pd.read_html(mars_facts_url)
# find total no. of tables in the list
len(tables)

2

In [24]:
# check and select table
mars_fact_df = tables[0]
mars_fact_df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [25]:
# add column names ['Description' , 'Mars', 'Earth']
mars_fact_df.columns = ['Description' , 'Mars', 'Earth']

# set Description as a index
mars_fact_df.set_index('Description' , drop=True , inplace= True)
mars_fact_df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [26]:
# Save the table in HTML format
table = mars_fact_df.to_html('table.html')

In [27]:
# Define the url and call create_soup function to return soup object
mars_hem_url = 'https://marshemispheres.com/'
soup = create_soup(mars_hem_url)

# Find the container that holds all the images in the main page
container = soup.find_all('div' , class_="item")

# Initialize the list
hemisphere_image_urls =[]

for content in container:    
    # use splinter to navigate to the page to get the image file 
    # Note: splinter uses main browser not the cotainer so be specific
    h3 = content.find('h3').text
    
    browser.find_by_text(str(h3)).click()
    
    # get html of rendered page and create a soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')   
    
    # Get the title and the image link
    title_hem = soup.find('h2' , class_="title").text
    
    image = mars_hem_url + soup.find('img', class_="wide-image")['src']
    #print(image)
    
    #create a dictionary
    hemisphere_image_urls.append({
                            'title' :title_hem,
                            'img_url':image
                             })

    # go back to the original page to grab more data
    browser.back()

In [28]:
# Close the browser
browser.quit()